# SI: Caffeine Solvation in Electrolyte Solutions
Stefan Hervø-Hansen<sup>a,b</sup>, Nobuyuki Matubayasi<sup>b,*</sup>, Mikael Lund<sup>a,*</sup>.<br><br>
<sup>a</sup> Division of Therotical Chemistry, Department of Chemistry, Lund University, Lund SE 221 00, Sweden.<br> <sup>b</sup> Division of Chemical Engineering, Graduate School of Engineering Science, Osaka University, Toyonaka, Osaka 560-8531, Japan.<br>
<sup>*</sup> To whom correspondence may be addressed nobuyuki@cheng.es.osaka-u.ac.jp and mikael.lund@teokem.lu.se.

## Introduction

We present a study of the solvation free energy of caffeine in electrolyte solutions using the energy representation description in combination with all-atom simulations. In specific we desire to calculate Setschenow (Sechenov) coefficients,$k_s$,[<sup>1</sup>](#fn1) for various salts. The Setschenow equation[<sup>1</sup>](#fn1) is on the $\ln$-scale given as
$$ \ln S/S_0 =  -k_sc_s = \ln\gamma = \Delta\mu^{ex} \tag 1$$
where $S$ and $S_0$ are solubilities in pure water and an electrolyte solution of concentration $c_s$. In litterature eq. 1 is most commonly presented on $\log_{10}$ scale, however on the persented form, the Setschenow equation shows its clear relationship to the activity coefficient and chemical potential. Finally we also desire to conduct a surface decomposition of caffeine to investigate the solvation free energy contribution for the different salts to the motifs of caffeine.

## Methods & Materials
Molecular dynamics simulations was conducted using the openMM (7.4.0)[<sup>2</sup>](#fn2) software package modded with the openmmtools[<sup>3</sup>](#fn3) and parmed[<sup>4</sup>](#fn4) packages. For the simulation of caffeine a GROMOS (ffGF53a6) derived Kirkwood-Buff force field with adjustments to the partial charges and geometrical parameters, which has previously been able able to reproduce experimental solubilities of caffeine in water and urea solutions[<sup>5</sup>](#fn5) was employed with the SPC/E[<sup>6</sup>](#fn6) force field employed for water and optimized ion parameters for alkali cations and halide anions[<sup>7</sup>](#fn7). The isothermal-isobaric ensemble was sampled using a combination of a geodesic Langevin integrator[<sup>8</sup>](#fn8) and a Monte Carlo barostat[<sup>9,</sup>](#fn9)[<sup>10</sup>](#fn10). The trajectories were analysed using MDtraj[<sup>11</sup>](#fn11) for structual properties, while ERmod[<sup>12</sup>](#fn12) was utilized for the calculation of solvation free energies.


## References
1. <span id="fn1"> Setschenow J (1889) Über die Konstitution der Salzlösungen auf Grund ihres Verhaltens zu Kohlensäure. Zeitschrift für Physikalische Chemie 4U(1):117–125.
2. <span id="fn1"> Eastman P, et al. (2017) OpenMM 7: Rapid development of high performance algorithms for molecular dynamics. PLOS Computational Biology 13(7):e1005659.</span><br>
3. <span id="fn2"> https://github.com/choderalab/openmmtools</span><br>
4. <span id="fn3"> https://github.com/ParmEd/ParmEd </span><br>
5. <span id="fn4"> Sanjeewa R, Weerasinghe S (2010) Development of a molecular mechanics force field for caffeine to investigate the interactions of caffeine in different solvent media. Journal of Molecular Structure: THEOCHEM 944(1–3):116–123. </span><br>
6. <span id="fn5"> Berendsen HJC, Grigera JR, Straatsma TP (1987) The missing term in effective pair potentials. The Journal of Physical Chemistry 91(24):6269–6271. </span><br>
7. <span id="fn6"> Heyda J, Vincent JC, Tobias DJ, Dzubiella J, Jungwirth P (2010) Ion Specificity at the Peptide Bond: Molecular Dynamics Simulations ofN-Methylacetamide in Aqueous Salt Solutions. The Journal of Physical Chemistry B 114(2):1213–1220. </span><br>
8. <span id="fn8"> Leimkuhler B, Matthews C (2016) Efficient molecular dynamics using geodesic integration and solvent–solute splitting. Proceedings of the Royal Society A: Mathematical, Physical and Engineering Sciences 472(2189):20160138. </span><br>
9. <span id="fn9"> Chow K-H, Ferguson DM (1995) Isothermal-isobaric molecular dynamics simulations with Monte Carlo volume sampling. Computer Physics Communications 91(1–3):283–289. </span><br>
10. <span id="fn10"> Åqvist J, Wennerström P, Nervall M, Bjelic S, Brandsdal BO (2004) Molecular dynamics simulations of water and biomolecules with a Monte Carlo constant pressure algorithm. Chemical Physics Letters 384(4–6):288–294. </span><br>
11. <span id="fn11"> McGibbon RT, et al. (2015) MDTraj: A Modern Open Library for the Analysis of Molecular Dynamics Trajectories. Biophysical Journal 109(8):1528–1532. </span><br>
12. <span id="fn12"> Sakuraba S, Matubayasi N (2014) Ermod: Fast and versatile computation software for solvation free energy with approximate theory of solutions. Journal of Computational Chemistry 35(21):1592–1608. </span><br>

### Import of python models

In [7]:
# Notebook dependent libs
import parmed as pmd
import math
import numpy as np
import matplotlib.pyplot as plt
import mdtraj as md
from uncertainties import unumpy
import os, time, re, yaml

from sklearn.decomposition import PCA   # For visual statisical analysis
from itertools import combinations

# Simulation specific libs
import sys
from simtk.openmm import app
import simtk.openmm as mm
import openmmtools as mmtools
import parmed as pmd
from mdtraj.reporters import XTCReporter

homedir='/home/stefan/Caffeine_solubility'

### Simulation settings

In [8]:
# State of simulations, (outFreq is steps per frame)
Nruns = 25
states = {                                                               
          'sol': {'Nsteps': 1000000,  'OutFreq': 50}, # 50 nanoseconds, 20.000 frames
          'ref': {'Nsteps': 200000,  'OutFreq': 500} # 0.4 nanoseconds, 400 frames
         }

salts = {'NaCl' : {'Cation': 'Na', 'Anion': 'Cl' },
         'NaI'  : {'Cation': 'Na', 'Anion': 'I'  },
         'NaSCN': {'Cation': 'Na', 'Anion': 'SCN'}
        }

# Concentrations of salts behaiving ideally
salt_concentrations = {0.00: {'Caffeine': 1, 'Water': 1000, 'Cation':0,  'Anion':0},
#                       0.25: {'Caffeine': 1, 'Water': 1000, 'Cation':5,  'Anion':5},
#                       0.50: {'Caffeine': 1, 'Water': 1000, 'Cation':9,  'Anion':9},
#                       0.75: {'Caffeine': 1, 'Water': 1000, 'Cation':14, 'Anion':14},
#                       1.00: {'Caffeine': 1, 'Water': 1000, 'Cation':18, 'Anion':18},
#                       1.25: {'Caffeine': 1, 'Water': 1000, 'Cation':23, 'Anion':23},   
#                       1.50: {'Caffeine': 1, 'Water': 1000, 'Cation':27, 'Anion':27},   
#                       1.75: {'Caffeine': 1, 'Water': 1000, 'Cation':32, 'Anion':32},
#                       2.00: {'Caffeine': 1, 'Water': 1000, 'Cation':36, 'Anion':36}   
                      }

### Construction of topology (.top) and structure (.pdb) files

In [11]:
packmol_script="""
tolerance 2.0
filetype pdb
output Caffeine_NaCl_sol_{run}.pdb
add_box_sides 0.5

structure ../../../../PDB_files/single-caffeine-molecule.pdb
        number {N_caffeine}
        fixed 16. 16. 16. 0. 0. 0.
        centerofmass
end structure

{salt}structure ../../../../PDB_files/Cl.pdb
{salt}        number {N_Cl}
{salt}        inside cube 0. 0. 0. 32.
{salt}end structure

{salt}structure ../../../../PDB_files/Na.pdb
{salt}        number {N_Na}
{salt}        inside cube 0. 0. 0. 32.
{salt}end structure

structure ../../../../PDB_files/water.pdb
        number {N_wat}
        inside cube 0. 0. 0. 32.
end structure
"""

topology="""
[ system ]
; Name
Caffeine in NaCl {conc} M aqueous solution.

[ defaults ]
; nbfunc        comb-rule       gen-pairs       fudgeLJ fudgeQQ
1               3               yes             0.5     0.8333

; Include all atomtypes
#include "/home/stefan/Caffeine_solubility/force_fields/atomtypes_spc.itp"
#include "/home/stefan/Caffeine_solubility/force_fields/atomtypes_ions.itp"
#include "/home/stefan/Caffeine_solubility/force_fields/atomtypes_caffeine.itp"
#include "/home/stefan/Caffeine_solubility/force_fields/atomtypes_scn.itp"

; Include all topologies
#include "/home/stefan/Caffeine_solubility/force_fields/ions.itp"
#include "/home/stefan/Caffeine_solubility/force_fields/spce.itp"
#include "/home/stefan/Caffeine_solubility/force_fields/caffeine-KBFF.itp"
#include "/home/stefan/Caffeine_solubility/force_fields/scn.itp"

[ molecules ]
; Compound         #mols
2S09               {N_caffeine}
{salt}Cl                 {N_Cl}
{salt}Na                 {N_Na}
SOL                {N_wat}
"""

In [12]:
%cd -q $homedir
for concentration, Nparticles in salt_concentrations.items():
    conc = '{0:.2f}'.format(concentration)
    %cd -q $homedir/Simulations/NaCl/$conc
    
    for run in range(Nruns):
        %cd -q $homedir/Simulations/NaCl/$conc/$run
        # Packmol Input
        with open('packmol.in', 'w') as text_file:
            # fix for no salt:
            if concentration:
                salt=''
            else:
                salt='#'
            text_file.write(packmol_script.format(N_caffeine=Nparticles['Caffeine'], N_wat=Nparticles['Water'],
                                                  N_Na=Nparticles['Cation'], N_Cl=Nparticles['Anion'],
                                                  salt=salt, run=run))
        !packmol < packmol.in

        # Topology input
        if run == 0:
            with open('../Caffeine_NaCl_sol.top', 'w') as text_file:
                # fix for no salt:
                if concentration:
                    salt=''
                else:
                    salt=';'
                text_file.write(topology.format(conc=concentration, salt=salt,
                                                N_caffeine=Nparticles['Caffeine'], N_wat=Nparticles['Water'],
                                                N_Na=Nparticles['Cation'], N_Cl=Nparticles['Anion']))
    
        # Solvated state
        mol = pmd.load_file('../Caffeine_NaCl_sol.top', xyz='Caffeine_NaCl_sol_{run}.pdb'.format(run=run))
        if run == 0:
            mol.save('../Caffeine_NaCl_sol.top', overwrite=True)
    
        # Reference state
        mol.strip(':2S09')
        if run == 0:
            mol.save('../Caffeine_NaCl_ref.top', overwrite=True)
        mol.save('Caffeine_NaCl_ref_{run}.pdb'.format(run=run)) 
    
    print('Wrote initial configurations and topology files to'+os.getcwd())


################################################################################

 PACKMOL - Packing optimization for the automated generation of
 starting configurations for molecular dynamics simulations.
 
                           Included as part of Packmol Memgen
                                                              Version 18.169 

################################################################################

  Packmol must be run with: packmol < inputfile.inp 

  Userguide at: www.ime.unicamp.br/~martinez/packmol 

  Reading input file... (Control-C aborts)
  Will print BOX SIDE informations. 
  Will sum   0.50000000000000000       to each side length on print
  Seed for random number generator:      1234567
  Output file: Caffeine_NaCl_sol_0.pdb
  Reading coordinate file: ../../../../PDB_files/single-caffeine-molecule.pdb
  Reading coordinate file: ../../../../PDB_files/water.pdb
  Number of independent structures:            2
  The structures are: 
  Structure  

/home/stefan/miniconda3/lib/python3.7/site-packages/parmed/gromacs/gromacstop.py:1025: GromacsWarning: 28 1-4 pairs were missing from the [ pairs ] section and were set to zero; make sure you know what you're doing!
  GromacsWarning)



################################################################################

 PACKMOL - Packing optimization for the automated generation of
 starting configurations for molecular dynamics simulations.
 
                           Included as part of Packmol Memgen
                                                              Version 18.169 

################################################################################

  Packmol must be run with: packmol < inputfile.inp 

  Userguide at: www.ime.unicamp.br/~martinez/packmol 

  Reading input file... (Control-C aborts)
  Will print BOX SIDE informations. 
  Will sum   0.50000000000000000       to each side length on print
  Seed for random number generator:      1234567
  Output file: Caffeine_NaCl_sol_1.pdb
  Reading coordinate file: ../../../../PDB_files/single-caffeine-molecule.pdb
  Reading coordinate file: ../../../../PDB_files/water.pdb
  Number of independent structures:            2
  The structures are: 
  Structure  

          |****************************

  Restraint-only function value:    1.8944030068206046E-002
  Maximum violation of the restraints:    3.8933169343439303E-003

--------------------------------------------------------------------------------

  Rescaling maximum and minimum coordinates... 
  Computing size of patches... 
  Add fixed molecules to permanent arrays... 
  Reseting center of mass... 

--------------------------------------------------------------------------------

  Setting initial trial coordinates ... 

--------------------------------------------------------------------------------

--------------------------------------------------------------------------------

  Molecules of type:            2
  Adjusting random positions to fit the constraints. 
  Packing:|0                                                             100%|
          |*******
  Restraint-only function value:    5.1504705777306281E-005
  Maximum violation of the restraints:    5.150461291597211

          |******************************************************************|
          |******************************************************************|


  Function value from last GENCAN loop: f = .34275E-02
  Best function value before: f = .17536E+05
  Improvement from best function value:    99.99 %
  Improvement from last loop:    99.99 %
  Maximum violation of target distance:     0.000000
  Maximum violation of the constraints: .22300E-02
  Current structure written to file: Caffeine_NaCl_sol_3.pdb
--------------------------------------------------------------------------------
  Packing solved for molecules of type           2
  Objective function value:    3.4274747327933904E-003
  Maximum violation of target distance:    0.0000000000000000     
  Max. constraint violation:    2.2300217163293734E-003
--------------------------------------------------------------------------------

################################################################################

  Packing

...

          |******************************************************************|
          |******************************************************************|


  Function value from last GENCAN loop: f = .34275E-02
  Best function value before: f = .17536E+05
  Improvement from best function value:    99.99 %
  Improvement from last loop:    99.99 %
  Maximum violation of target distance:     0.000000
  Maximum violation of the constraints: .22300E-02
  Current structure written to file: Caffeine_NaCl_sol_19.pdb
--------------------------------------------------------------------------------
  Packing solved for molecules of type           2
  Objective function value:    3.4274747327933904E-003
  Maximum violation of target distance:    0.0000000000000000     
  Max. constraint violation:    2.2300217163293734E-003
--------------------------------------------------------------------------------

################################################################################

  Packin


################################################################################

 PACKMOL - Packing optimization for the automated generation of
 starting configurations for molecular dynamics simulations.
 
                           Included as part of Packmol Memgen
                                                              Version 18.169 

################################################################################

  Packmol must be run with: packmol < inputfile.inp 

  Userguide at: www.ime.unicamp.br/~martinez/packmol 

  Reading input file... (Control-C aborts)
  Will print BOX SIDE informations. 
  Will sum   0.50000000000000000       to each side length on print
  Seed for random number generator:      1234567
  Output file: Caffeine_NaCl_sol_21.pdb
  Reading coordinate file: ../../../../PDB_files/single-caffeine-molecule.pdb
  Reading coordinate file: ../../../../PDB_files/water.pdb
  Number of independent structures:            2
  The structures are: 
  Structure 

          |****************************

  Restraint-only function value:    1.8944030068206046E-002
  Maximum violation of the restraints:    3.8933169343439303E-003

--------------------------------------------------------------------------------

  Rescaling maximum and minimum coordinates... 
  Computing size of patches... 
  Add fixed molecules to permanent arrays... 
  Reseting center of mass... 

--------------------------------------------------------------------------------

  Setting initial trial coordinates ... 

--------------------------------------------------------------------------------

--------------------------------------------------------------------------------

  Molecules of type:            2
  Adjusting random positions to fit the constraints. 
  Packing:|0                                                             100%|
          |*******
  Restraint-only function value:    5.1504705777306281E-005
  Maximum violation of the restraints:    5.150461291597211

          |******************************************************************|
          |******************************************************************|


  Function value from last GENCAN loop: f = .34275E-02
  Best function value before: f = .17536E+05
  Improvement from best function value:    99.99 %
  Improvement from last loop:    99.99 %
  Maximum violation of target distance:     0.000000
  Maximum violation of the constraints: .22300E-02
  Current structure written to file: Caffeine_NaCl_sol_23.pdb
--------------------------------------------------------------------------------
  Packing solved for molecules of type           2
  Objective function value:    3.4274747327933904E-003
  Maximum violation of target distance:    0.0000000000000000     
  Max. constraint violation:    2.2300217163293734E-003
--------------------------------------------------------------------------------

################################################################################

  Packin

Wrote initial configurations and topology files to/home/stefan/Caffeine_solubility/Simulations/NaCl/0.00/24


## Molecular dynamics simulations
### Simulation setup using OpenMM

In [13]:
%cd -q $homedir
N_simulations = 0

for concentration in salt_concentrations:
    conc = '{0:.2f}'.format(concentration)
    %cd -q $homedir/Simulations/NaCl/$conc
    
    for run in range(Nruns):
        %cd -q $homedir/Simulations/NaCl/$conc/$run
    
        for state, settings in states.items():
        
            openmm_script="""
# Imports
import sys
import os
from simtk.openmm import app
import simtk.openmm as mm
import openmmtools as mmtools
from parmed import load_file, unit as u
from mdtraj.reporters import XTCReporter

print('Loading initial configuration and toplogy')
init_conf = load_file('../Caffeine_NaCl_{state}.top', xyz='Caffeine_NaCl_{state}_{run}.pdb')

# Creating system
print('Creating OpenMM System')
system = init_conf.createSystem(nonbondedMethod=app.PME, ewaldErrorTolerance=0.00001,
                                nonbondedCutoff=1.2*u.nanometers, constraints=app.HBonds)

# Calculating total mass of system
total_mass = 0
for i in range(system.getNumParticles()):
    total_mass += system.getParticleMass(i).value_in_unit(u.dalton)
total_mass *= u.dalton

# Temperature-coupling by geodesic Langevin integrator (NVT)
integrator = mmtools.integrators.GeodesicBAOABIntegrator(K_r = 3,
                                                         temperature = 298.15*u.kelvin,
                                                         collision_rate = 1.0/u.picoseconds,
                                                         timestep = 2.0*u.femtoseconds
                                                        )

# Pressure-coupling by a Monte Carlo Barostat (NPT)
system.addForce(mm.MonteCarloBarostat(1*u.bar, 298.15*u.kelvin, 25))

# Add LJ switching functions
forces = {{system.getForce(index).__class__.__name__: 
          system.getForce(index) for index in range(system.getNumForces())}}
forces['CustomNonbondedForce'].setUseSwitchingFunction(True)
forces['CustomNonbondedForce'].setSwitchingDistance(1*u.nanometer)

platform = mm.Platform.getPlatformByName('CUDA')
properties = {{'CudaPrecision': 'mixed', 'CudaDeviceIndex': '0'}}

# Create the Simulation object
sim = app.Simulation(init_conf.topology, system, integrator, platform, properties)

# Set the particle positions
sim.context.setPositions(init_conf.positions)

# Minimize the energy
print('Minimizing energy')
sim.minimizeEnergy(tolerance=1*u.kilojoule/u.mole, maxIterations=500000)
    
# Draw initial MB velocities
sim.context.setVelocitiesToTemperature(298.15*u.kelvin)

# Equlibrate simulation
print('Equilibrating...')
sim.step(50000)  # 50000*2 fs = 0.1 ns

# Print PME information
print('''
PARTICLE MESH EWALD PARAMETERS (Production run)
Separation parameter: {{}}
Number of grid points along the X axis: {{}}
Number of grid points along the Y axis: {{}}
Number of grid points along the Z axis: {{}}
'''.format(*forces['NonbondedForce'].getPMEParametersInContext(sim.context)))

# Set up the reporters
sim.reporters.append(app.StateDataReporter('output_{state}.dat', {outFreq}, totalSteps={Nsteps}+50000,
    time=True, potentialEnergy=True, kineticEnergy=True, temperature=True, volume=True, density=True,
    systemMass=total_mass, remainingTime=True, speed=True, separator='\t'))

# Set up trajectory reporter
sim.reporters.append(XTCReporter('trajectory_{state}.xtc', reportInterval={outFreq}, append=False))

# Run dynamics
print('Running dynamics! (NPT)')
sim.step({Nsteps})
"""
            with open('openMM_{state}.py'.format(state=state), 'w') as text_file:
                text_file.write(openmm_script.format(state=state, Nsteps=settings['Nsteps'], outFreq=settings['OutFreq'],
                                                     run=run))
                N_simulations+=1
        print('Wrote run_openMM.py files to '+os.getcwd())
    
print('Simulations about to be submitted: {}'.format(N_simulations))

Wrote run_openMM.py files to /home/stefan/Caffeine_solubility/Simulations/NaCl/0.00/0
Wrote run_openMM.py files to /home/stefan/Caffeine_solubility/Simulations/NaCl/0.00/1
Wrote run_openMM.py files to /home/stefan/Caffeine_solubility/Simulations/NaCl/0.00/2
Wrote run_openMM.py files to /home/stefan/Caffeine_solubility/Simulations/NaCl/0.00/3
Wrote run_openMM.py files to /home/stefan/Caffeine_solubility/Simulations/NaCl/0.00/4
Wrote run_openMM.py files to /home/stefan/Caffeine_solubility/Simulations/NaCl/0.00/5
Wrote run_openMM.py files to /home/stefan/Caffeine_solubility/Simulations/NaCl/0.00/6
Wrote run_openMM.py files to /home/stefan/Caffeine_solubility/Simulations/NaCl/0.00/7
Wrote run_openMM.py files to /home/stefan/Caffeine_solubility/Simulations/NaCl/0.00/8
Wrote run_openMM.py files to /home/stefan/Caffeine_solubility/Simulations/NaCl/0.00/9
Wrote run_openMM.py files to /home/stefan/Caffeine_solubility/Simulations/NaCl/0.00/10
Wrote run_openMM.py files to /home/stefan/Caffeine_so

### Submit script

In [15]:
#for concentration in salt_concentrations:
#    conc = '{0:.2f}'.format(concentration)
#    %cd -q $homedir/Simulations/NaCl/$conc
    
for run in range(Nruns):
        #%cd -q $homedir/Simulations/NaCl/$conc/$run
    %cd -q $homedir/Simulations/Nosalt/$run
    
    for state in states:
        
        submit_script="""#!/bin/bash
#PBS -l nodes=1:ppn=18:gpu:gpus=1     # 1 node, 18 cores, GPU node, 1 gpu.
#PBS -N {conc}_M_NaCl_{state}_{run}   # Name of job
#PBS -e run_{state}.err               # error output
#PBS -o run_{state}.out               # output file name

source ~/.bashrc
source ~/.bash_profile
cd {path}

python openMM_{state}.py"""

        with open('submit_{state}.pbs'.format(state=state), 'w') as text_file:
                text_file.write(submit_script.format(conc=conc, state=state, run=run, path=os.getcwd()))
        #!qsub submit_sol.pbs
        time.sleep(1) # Safety in submission of jobs: can cause problems if too fast
        #!qsub submit_ref.pbs
        time.sleep(1) # Safety in submission of jobs: can cause problems if too fast

### Trajectory processing

In [ ]:
for concentration in salt_concentrations:
    for state in states:
        conc = '{0:.2f}'.format(concentration)
        %cd -q $homedir/Simulations/NaCl/$conc
    
        for run in range(Nruns):
            if run == 0:
                traj = md.load_xtc('0/trajectory_{state}.xtc'.format(state=state),
                                   top='Caffeine_NaCl_{state}.xtc'.format(state=state))
                continue
            traj_app = md.load_xtc('{run}/trajectory_{state}.xtc'.format(run=run, state=state)
                                   top='Caffeine_NaCl_{state}.xtc'.format(state=state))
            traj = md.join([traj, traj_app], check_topology=True, discard_overlapping_frames=False)
        
        traj.save_xtc('trajectory_{state}_full.xtc'.format(state=state), force_overwrite=False)

### Prepare ERmod analysis
<img style="float: right;width:319.5px;height:312.5px;" src="images/ermod.png" title="Simulation flow" >

The determination of solvation free energies and chemical potentials though the ERmod software is conducted in two stages as illustrated in the figure. First is the determination of the energy distribution functions from the simulation trajectories obtained from molecular dynamics using the subprogram `ermod`. Second is the solvation free energy is determined from the energy distribution functions though an approximate functional using the subprogram `slvfe`.

Documentation for the parameter file `parameters_er` which determines how the `ermod` subprogram runs can be found [here](https://sourceforge.net/p/ermod/wiki/parameters-ermod03/). In the python code below it can be noted many of the parameters has been given the keyword `[correct]`, these settings are dependent on the simulation settings and or the software being used and are thus recommended not to be changed if using the openMM setup above. The parameters given the keyword `[to be set]` are settings either collected from the output of the openMM script or set by the user, in specific the values for `maxins` and `engdiv` should be given by the user are given the default value of 1000 and 5 respectively.

Documentation for the parameter file `parameters_fe` which determines how the `slvfe` subprogram runs can be found [here](https://sourceforge.net/p/ermod/wiki/parameters-slvfe/). It is recommended not to change the parameters in this file with the exception of the temperature if conducting simulations at other temperatures than 298.15 Kelvin, and the volume. In the code below the average volume input is calculated as an average volume from the two ensemble volume averages.

In [ ]:
# ERmod settings should be moved here
Ninserts=1000

ermod_script = """
&ene_param
      slttype = {state},     ! Choose system; 1: Solution system  2,3: Reference                     [to be set]
      boxshp = 1,            ! Boxtype                                                               [correct]   
      estype = 2,            ! Ensemble; 1: NVT   2: NPT                                             [correct]   
      inptemp = 298.15,      ! Temperature (in Kelvin)                                               [correct]   
      ljformat = 2,          ! LJ form                                                               [correct]   
      cmbrule = 1,           ! Combination rule                                                      [correct]   
      ljswitch = 1,          ! Switching function for smooth LJ truncation                           [correct]   
      upljcut = 12,          ! Upper limit for LJ cutoff switching function (in Angstrom)            [correct]   
      lwljcut = 10,          ! Lower limit for LJ cutoff switching function (in Angstrom)            [correct]   
      cltype = 2,            ! Treatment of Coulomb interaction (2=PME)                              [correct]   
      elecut = 12.0,         ! Cutoff of the real-space electrostatic interaction (in angstrom)      [correct]   
      ewtoler = 0.00001,     ! Error tolerance in Ewald                                              [correct]   
      splodr = 5,            ! Order of spline function used in PME                                  [correct]   
      ms1max = {PMEnodes_x}, ! Number of meshes in PME (x)                                           [to be set] 
      ms2max = {PMEnodes_y}, ! Number of meshes in PME (y)                                           [to be set] 
      ms3max = {PMEnodes_z}, ! Number of meshes in PME (z)                                           [to be set] 
{ref} maxins = {Ninserts},   ! Number of inserts (chosen freely, for reference ONLY)                 [to be set] 
      engdiv = {Ndivisions}, ! Number of divisions of the total simulation length (chosen freely)    [to be set] 
/
&hist
      eclbin=5.0e-2, ecfbin=2.0e-3, ec0bin=2.0e-4, finfac=10.0e0,
      ecdmin=-35.000000, ecfmns=-0.20e0, ecdcen=0.0e0, eccore=20.0e0,
      ecdmax=1.0e11, pecore=200
/
"""
    
slvfe_script = """
&fevars
clcond    = "merge",       ! Calculation type, "merge" / "basic" / "range"
numsln    = {Nsol_blocks}, ! Number of trajectory blocks in solution system
numref    = {Nref_blocks}, ! Number of trajectory blocks in reference system
numdiv    = {Nsol_blocks}, ! Number of division for statistics, usually set equal to numsln
avevolume = {avgV},        ! Average volume sampled in simulation (in Angstrom^3)

ljlrc     = 'yes',         ! Long-range correction of the Lennard-Jones interaction (avevolume must be specified)
uvread    = "yes",         ! "not" if average solute-solvent energy is calculated from engsln, instead of aveuv.tt
slfslt    = "yes",         ! "not" if the solute self energy is not read
infchk    = "yes",         ! Enable error analysis for the logarithmic-mesh part
inptemp   = 298.15,        ! Input Temperature in Kelvin
/
"""

### Generate ERmod Files

In [ ]:
%cd -q $homedir

for concentration in salt_concentrations:
    conc = '{0:.2f}'.format(concentration)
    %cd -q $homedir/Simulations/NaSCN/$conc
    !echo 2S09 | /usr/bin/python ~/software/ermod-0.3.6_openMM/share/ermod/tools/gromacs/gen_structure --top Caffeine_NaSCN_sol.top
    
    #!# Solution settings #!#
    %cd -q $homedir/Simulations/NaSCN/$conc/soln
    
    ###### WRITING ERMOD PARAMETER FILE ######
    # PME settings
    with open('../run_sol.out', 'r') as logfile:
        lines = logfile.readlines()
        logfile.close()
    index = [idx for idx, s in enumerate(lines) if 'PARTICLE MESH EWALD PARAMETERS' in s][0]
    PME_spacing = re.findall("([0-9]+[,.]+[0-9]+)", lines[index+1])
    N_gridpoints = (*re.findall("([0-9]+)", lines[index+2]), *re.findall("([0-9]+)", lines[index+3]), *re.findall("([0-9]+)", lines[index+4]))
    
    comment = '!'
    stateID = 1
    
    with open('parameters_er', 'w') as text_file:
        text_file.write(ermod_script.format(state=stateID, Ninserts=1000, Ndivisions=50, PMEnodes_x=N_gridpoints[0],
                                            PMEnodes_y=N_gridpoints[1], PMEnodes_z=N_gridpoints[2], ref=comment))
    
    ####### CORRECTING MDinfo FILE #########
    with open('MDinfo', 'r') as f:
        lines = f.readlines()
        f.close()
        
    with open('MDinfo', 'w') as f:
        lines[0] = lines[0].replace('FRAMES', '500000')
        for line in lines:
            f.write(line)
        f.close()
    
    ####### CREATING HISTORY SYMLINK #########
    !ln -s $homedir/Simulations/NaSCN/$conc/trajectory_sol.xtc HISTORY
    
    
    
    #!# Reference settings #!#
    %cd -q $homedir/Simulations/NaSCN/$conc/refs
    
    ###### WRITING ERMOD PARAMETER FILE ######
    # PME settings
    with open('../run_ref.out', 'r') as logfile:
        lines = logfile.readlines()
        logfile.close()
    index = [idx for idx, s in enumerate(lines) if 'PARTICLE MESH EWALD PARAMETERS' in s][0]
    PME_spacing = re.findall("([0-9]+[,.]+[0-9]+)", lines[index+1])
    N_gridpoints = (*re.findall("([0-9]+)", lines[index+2]), *re.findall("([0-9]+)", lines[index+3]), *re.findall("([0-9]+)", lines[index+4]))
    
    comment = '  '
    stateID = 3     # 3=flexible, 1=fixed
    
    with open('parameters_er', 'w') as text_file:
        text_file.write(ermod_script.format(state=stateID, Ninserts=1000, Ndivisions=10, PMEnodes_x=N_gridpoints[0],
                                            PMEnodes_y=N_gridpoints[1], PMEnodes_z=N_gridpoints[2], ref=comment))
    
    ####### CORRECTING MDinfo FILE #########
    with open('MDinfo', 'r') as f:
        lines = f.readlines()
        f.close()
        
    with open('MDinfo', 'w') as f:
        lines[0] = lines[0].replace('FRAMES', '10000')
        for line in lines:
            f.write(line)
        f.close()
    
    ####### CREATING HISTORY AND SltConf SYMLINK #########
    !ln -s $homedir/Simulations/NaSCN/$conc/trajectory_ref.xtc HISTORY
    !ln -s $homedir/Simulations/Solute/trajectory_solute.xtc SltConf
    
    ####### COPY SOLUTE CONFIGURATION FILE INTO CURRENT DIR ##########
    #%cp $homedir/SltInfo .                     # fixed configuration
    
    
    #! FREE ENERGY CALCULATION #!#
    %cd -q $homedir/Simulations/NaSCN/$conc
    # calculate avg volume from the two ensemble averages
    vol_ref = np.loadtxt('output_ref.dat', usecols=4, skiprows=1)
    vol_sol = np.loadtxt('output_sol.dat', usecols=4, skiprows=1)
    avgVolume = (vol_ref.mean()*1000 + vol_sol.mean()*1000)/2
    
    # Make parameter file
    with open('parameters_fe', 'w') as text_file:
        text_file.write(slvfe_script.format(Nsol_blocks=50, Nref_blocks=10, avgV=avgVolume))
        
    print('Wrote ERmod analysis files to '+os.getcwd())

### Submit ERmod analysis

In [ ]:
submit_script="""#!/bin/bash
#PBS -l nodes=1:ppn=36:cpu            # 1 node, 36 cores, CPU nodes
#PBS -N ermod_{conc}                  # Name of job
#PBS -e ermod.err                     # error output
#PBS -o ermod.out                     # output file name

source ~/.bashrc
source ~/.bash_profile
cd {path}

# Solution step
cd soln
mpirun -np 36 ermod
cd ..

# Reference step
cd refs
mpirun -np 36 ermod
cd ..

# Calculate solvation energy
slvfe
"""


%cd -q $homedir
for i, concentration in enumerate(salt_concentrations):
    conc = '{0:.2f}'.format(concentration)
    %cd -q $homedir/Simulations/NaSCN/$conc
    
    with open('submit_ermod.pbs', 'w') as text_file:
            text_file.write(submit_script.format(path=os.getcwd(), conc=conc))
    !qsub submit_ermod.pbs
    time.sleep(1)

## Load and plot solvation free energies

The results from the ERmod analysis is stored in the file `_results.yml` which we now load and plot. The results are obtained from 50 ns long MD simulations of caffeine in electrolyte solutions.
We see that both NaCl and NaI leads to *salting out*, i.e. the solvation free energy of caffeine is increased compared to pure water.

In [ ]:
%cd -q $homedir
kcal_to_kT = 1.688 # Assuming 298.15 K

with open('Simulations/_results.yml') as f: # open structured result file (YAML)
    
    r = yaml.load(f, Loader=yaml.Loader)
    
    fig, (ax1, ax2) = plt.subplots(1,2, sharex=True)
    fig.set_size_inches(10,4.5)
    fig.tight_layout()
    
    for d in r['salts']: # loop over all salt types
        conc   = np.array(d['conc'])                       # molar conc.
        mu     = unumpy.uarray(d['muexcess'], d['error'])  # excess chem. pot. and error
        mu0    = mu[0]                                     # in pure water
        gamma  = unumpy.exp( (mu-mu0)*kcal_to_kT )         # activity coefficient
        fit    = np.polyfit(conc, unumpy.nominal_values(mu), 1, w=1/unumpy.std_devs(mu))
        fit_fn = np.poly1d(fit)
        print("Setschenow coefficient ("+d['label']+") =", fit[0])
        ax1.errorbar(conc, unumpy.nominal_values(mu), yerr=unumpy.std_devs(mu),
                     fmt='o', c=d['color'], alpha=0.6, ms=10)
        ax1.plot(conc, fit_fn(conc), label=d['label'], c=d['color'], lw=2, ms=10)
        ax2.errorbar(conc, unumpy.nominal_values(gamma), yerr=unumpy.std_devs(gamma),
                     label=d['label'], c=d['color'], fmt='o', ms=10, linestyle=None)

    ax1.set_title('Solvation Free Energy')
    ax2.set_title('Activity Coefficient')
    ax1.set_xlabel('Salt concentration (mol/l)')
    ax1.set_ylabel('$\Delta G_{sol}$ (kcal/mol)')
    ax2.set_xlabel('Salt concentration (mol/l)')
    ax2.set_ylabel('$\gamma$')
    ax1.legend(loc=2, frameon=False, fontsize='large')
plt.savefig('images/solvation.pdf', bbox_inches='tight')
plt.show()

In [ ]:
%cd -q $homedir

with open('Simulations/_results.yml') as f: # open structured result file (YAML)
    r = yaml.load(f, Loader=yaml.Loader)
    fig, ax = plt.subplots()
    fig.set_size_inches(6,6)
    width=0.3
    offset=0
    index = np.arange(4)
    
    for d in r['salts']: # loop over all salt types
        mu = unumpy.uarray(d['decomposition'][0]['mu'], d['decomposition'][0]['error'])
        ax.bar( index+offset, unumpy.nominal_values(mu),
                color=d['color'], label=d['label'], width=width,
                alpha=0.7, yerr=unumpy.std_devs(error), capsize=2)
        offset = offset + width
ax.set_xticks(index + width / 2)
ax.set_xticklabels(('Total', 'Water', 'Cations', 'Anions'))
ax.set_ylabel('$\Delta G_{sol}$ (kcal/mol)')
#ax.set_title('Free energy decomposition (2 M)')
ax.legend(loc=0, frameon=False, fontsize='large')
#plt.savefig('images/solvent-decomposition.pdf', bbox_inches='tight')
plt.show()

## Decomposition by Caffeine Motifs

Todo...

In [ ]:
%cd -q $homedir
mol = md.load_xtc('Simulations/old_simulations/NaCl/2.00/trajectory_sol.xtc',
                  top='Simulations/old_simulations/NaCl/2.00/Caffeine_NaCl_sol.pdb')

In [ ]:
methyl1_wat = mol.topology.select_pairs(selection1="index 0", selection2="water and element O")
methyl2_wat = mol.topology.select_pairs(selection1="index 16", selection2="water and element O")
methyl3_wat = mol.topology.select_pairs(selection1="index 22", selection2="water and element O")
ketone1_wat = mol.topology.select_pairs(selection1="index 15", selection2="water and element O")
ketone2_wat = mol.topology.select_pairs(selection1="index 21", selection2="water and element O")
imine_wat   = mol.topology.select_pairs(selection1="index 7", selection2="water and element O")
aromatic1_wat = mol.topology.select_pairs(selection1="index 4", selection2="water and element O")
aromatic2_wat = mol.topology.select_pairs(selection1="index 5", selection2="water and element O")


methyl1_Cl   = mol.topology.select_pairs(selection1="index 0",  selection2="element Cl")
methyl2_Cl   = mol.topology.select_pairs(selection1="index 16", selection2="element Cl")
methyl3_Cl   = mol.topology.select_pairs(selection1="index 22", selection2="element Cl")
ketone1_Cl   = mol.topology.select_pairs(selection1="index 15", selection2="element Cl")
ketone2_Cl   = mol.topology.select_pairs(selection1="index 21", selection2="element Cl")
imine_Cl     = mol.topology.select_pairs(selection1="index 7",  selection2="element Cl")
aromatic1_Cl = mol.topology.select_pairs(selection1="index 4",  selection2="element Cl")
aromatic2_Cl = mol.topology.select_pairs(selection1="index 5",  selection2="element Cl")


methyl1_Na   = mol.topology.select_pairs(selection1="index 0", selection2="resname SOD")
methyl2_Na   = mol.topology.select_pairs(selection1="index 16", selection2="resname SOD")
methyl3_Na   = mol.topology.select_pairs(selection1="index 22", selection2="resname SOD")
ketone1_Na   = mol.topology.select_pairs(selection1="index 15", selection2="resname SOD")
ketone2_Na   = mol.topology.select_pairs(selection1="index 21", selection2="resname SOD")
imine_Na     = mol.topology.select_pairs(selection1="index 7", selection2="resname SOD")
aromatic1_Na = mol.topology.select_pairs(selection1="index 4", selection2="resname SOD")
aromatic2_Na = mol.topology.select_pairs(selection1="index 5", selection2="resname SOD")

In [ ]:
rdf_methyl1_wat = md.compute_rdf(mol, methyl1_wat)
rdf_methyl2_wat = md.compute_rdf(mol, methyl2_wat)
rdf_methyl3_wat = md.compute_rdf(mol, methyl3_wat)
rdf_ketone1_wat = md.compute_rdf(mol, ketone1_wat)
rdf_ketone2_wat = md.compute_rdf(mol, ketone2_wat)
rdf_imine_wat   = md.compute_rdf(mol, imine_wat)
rdf_aromatic1_wat = md.compute_rdf(mol, aromatic1_wat)
rdf_aromatic2_wat = md.compute_rdf(mol, aromatic2_wat)

rdf_methyl1_Cl = md.compute_rdf(mol, methyl1_Cl)
rdf_methyl2_Cl = md.compute_rdf(mol, methyl2_Cl)
rdf_methyl3_Cl = md.compute_rdf(mol, methyl3_Cl)
rdf_ketone1_Cl = md.compute_rdf(mol, ketone1_Cl)
rdf_ketone2_Cl = md.compute_rdf(mol, ketone2_Cl)
rdf_imine_Cl   = md.compute_rdf(mol, imine_Cl)
rdf_aromatic1_Cl = md.compute_rdf(mol, aromatic1_Cl)
rdf_aromatic2_Cl = md.compute_rdf(mol, aromatic2_Cl)

rdf_methyl1_Na = md.compute_rdf(mol, methyl1_Na)
rdf_methyl2_Na = md.compute_rdf(mol, methyl2_Na)
rdf_methyl3_Na = md.compute_rdf(mol, methyl3_Na)
rdf_ketone1_Na = md.compute_rdf(mol, ketone1_Na)
rdf_ketone2_Na = md.compute_rdf(mol, ketone2_Na)
rdf_imine_Na   = md.compute_rdf(mol, imine_Na)
rdf_aromatic1_Na = md.compute_rdf(mol, aromatic1_Na)
rdf_aromatic2_Na = md.compute_rdf(mol, aromatic2_Na)

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(12, 8))


axes[0][0].plot(rdf_methyl1_wat[0], rdf_methyl1_wat[1], label='Methyl 1')
axes[0][0].plot(rdf_methyl2_wat[0], rdf_methyl2_wat[1], label='Methyl 2')
axes[0][0].plot(rdf_methyl3_wat[0], rdf_methyl3_wat[1], label='Methyl 3')
axes[0][0].vlines(0.23, -0.1, 0.1)
axes[0][0].vlines(0.58, 0.7, 0.9)
axes[0][0].vlines(0.8, 1.1, 0.9)
axes[0][0].legend(loc=0)
axes[0][0].set_title('Methyl Groups', pad=10)
axes[0][0].set_ylabel('Water', rotation=0, size='large', labelpad=25)
axes[0][1].plot(rdf_ketone1_wat[0], rdf_ketone1_wat[1], label='Ketone 1')
axes[0][1].plot(rdf_ketone2_wat[0], rdf_ketone2_wat[1], label='Ketone 2')
axes[0][1].plot(rdf_imine_wat[0], rdf_imine_wat[1], label='Imine')
axes[0][1].vlines(0.13, -0.1, 0.1)
axes[0][1].vlines(0.255, 0.0, 0.2)
axes[0][1].vlines(0.35, 0.5, 0.7)
axes[0][1].vlines(0.52, 0.7, 0.9)
axes[0][1].vlines(0.70, 1.1, 0.9)
axes[0][1].legend(loc=0)
axes[0][1].set_title('Polar Groups', pad=10)
axes[0][2].plot(rdf_aromatic1_wat[0], rdf_aromatic1_wat[1], label='Carbon 1')
axes[0][2].plot(rdf_aromatic2_wat[0], rdf_aromatic2_wat[1], label='Carbon 2')
axes[0][2].vlines(0.25, -0.1, 0.1)
axes[0][2].vlines(0.435, 0.6, 0.7)
axes[0][2].vlines(0.62, 0.9, 1.1)
axes[0][2].vlines(0.79, 0.9, 1.1)
axes[0][2].legend(loc=0)
axes[0][2].set_title('Shared aromatic carbon', pad=10)


axes[1][0].plot(rdf_methyl1_Cl[0], rdf_methyl1_Cl[1], label='Methyl 1')
axes[1][0].plot(rdf_methyl2_Cl[0], rdf_methyl2_Cl[1], label='Methyl 2')
axes[1][0].plot(rdf_methyl3_Cl[0], rdf_methyl3_Cl[1], label='Methyl 3')
axes[1][0].legend(loc=0)
axes[1][0].set_ylabel('Cl', rotation=0, size='large', labelpad=15)
axes[1][1].plot(rdf_ketone1_Cl[0], rdf_ketone1_Cl[1], label='Ketone 1')
axes[1][1].plot(rdf_ketone2_Cl[0], rdf_ketone2_Cl[1], label='Ketone 2')
axes[1][1].plot(rdf_imine_Cl[0], rdf_imine_Cl[1], label='Imine')
axes[1][1].legend(loc=0)
axes[1][2].plot(rdf_aromatic1_Cl[0], rdf_aromatic1_Cl[1], label='Carbon 1')
axes[1][2].plot(rdf_aromatic2_Cl[0], rdf_aromatic2_Cl[1], label='Carbon 2')
axes[1][2].legend(loc=0)

axes[2][0].plot(rdf_methyl1_Na[0], rdf_methyl1_Na[1], label='Methyl 1')
axes[2][0].plot(rdf_methyl2_Na[0], rdf_methyl2_Na[1], label='Methyl 2')
axes[2][0].plot(rdf_methyl3_Na[0], rdf_methyl3_Na[1], label='Methyl 3')
axes[2][0].legend(loc=0)
axes[2][0].set_ylabel('Na', rotation=0, size='large', labelpad=15)
axes[2][1].plot(rdf_ketone1_Na[0], rdf_ketone1_Na[1], label='Ketone 1')
axes[2][1].plot(rdf_ketone2_Na[0], rdf_ketone2_Na[1], label='Ketone 2')
axes[2][1].plot(rdf_imine_Na[0], rdf_imine_Na[1], label='Imine')
axes[2][1].legend(loc=0)
axes[2][2].plot(rdf_aromatic1_Na[0], rdf_aromatic1_Na[1], label='Carbon 1')
axes[2][2].plot(rdf_aromatic2_Na[0], rdf_aromatic2_Na[1], label='Carbon 2')
axes[2][2].legend(loc=0)

plt.tight_layout()

In [ ]:
volume = np.loadtxt('../../NaCl/2.00/output_sol.dat', usecols=4, skiprows=1, unpack=True)

In [ ]:
print(volume.mean())

In [ ]:
test_NaSCN = [-16.2172, -16.0884, -16.2405, -16.1340, -16.0941, -16.0424, -16.1017, -16.0310, -16.0011, -15.9893,
              -16.0517, -16.1674, -16.2814, -16.0788, -16.0925, -16.2353, -16.1456, -16.0307, -16.0687, -16.1101,
              -16.0148, -16.0083, -16.0674, -16.1539, -16.1924, -16.1332, -16.1846, -16.0680, -16.1272, -16.0153,
              -15.9826, -16.1589, -16.0629, -15.9631, -16.0645, -16.1726, -16.0302, -16.1413, -16.0443, -16.1847,
              -16.1754, -16.2207, -16.1465, -16.0193, -16.1537, -15.9654, -16.1147, -16.1920, -16.1875, -15.9987]

test_NaCl  = [-16.0137, -16.2569, -16.2331, -16.0739, -16.2546, -16.1352, -16.0440, -16.1728, -16.0714, -16.1831,
              -16.0691, -16.1240, -16.0893, -16.0717, -16.0627, -16.2194, -16.1050, -16.0789, -15.9744, -16.2921,
              -15.9701, -16.1681, -16.1635, -16.1864, -16.2210, -16.0491, -16.1407, -16.1883, -16.1295, -16.1614,
              -16.1309, -16.2221, -15.8936, -16.2133, -16.0457, -16.1059, -16.1310, -16.0572, -15.9356, -16.1874,
              -16.1247, -16.0512, -16.1340, -16.3074, -16.1454, -16.1306, -16.0821, -16.0655, -16.1155, -16.0353]

test_NaI   = [-16.0770, -16.0244, -16.1092, -16.2013, -16.0709, -15.9253, -16.0682, -15.9637, -15.8816, -16.2146,
              -16.1554, -15.9604, -15.9419, -16.0009, -16.0603, -15.9381, -16.0457, -15.9122, -15.9016, -16.0051,
              -16.0847, -16.0389, -15.9882, -16.0832, -16.1091, -16.0965, -16.0418, -16.0737, -16.0929, -16.1038,
              -16.1218, -16.0040, -16.0627, -16.0539, -16.0172, -16.2199, -16.0819, -16.1423, -16.0846, -16.1014,
              -16.0096, -16.0522, -16.1390, -16.0189, -16.1025, -16.1159, -16.1431, -16.0713, -16.0123, -16.0250]


hist_1, bins = np.histogram(test_nASCN, bins=12, range=(-16.5, -15.7), density=True)
hist_2, bins = np.histogram(test_NaCl, bins=12, range=(-16.5, -15.7), density=True)
hist_3, bins = np.histogram(test_NaI, bins=12, range=(-16.5, -15.7), density=True)

means = []
for i in range(len(bins[:-1])):
    means.append((bins[i]+bins[i+1])/2)
    

In [ ]:
plt.plot(means, hist_1, label='NaSCN')  # mean: -16.1029 +/- 0.0226
plt.plot(means, hist_2, label='NaCl')   # mean: -16.1204 +/- 0.0247
plt.plot(means, hist_3, label='NaI')    # mean: -16.0550 +/- 0.0218
plt.ylabel('Density')
plt.xlabel('This is now latex $\Delta \mu^{\mathrm{excess}}$ asd $\mu$')
plt.legend(loc=0)

plt.savefig('images/distribution.png', dpi=300)
plt.show()


plt.tight_layout()

